In [2]:
from langchain.document_loaders import PyPDFLoader

In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from transformers import GPT2TokenizerFast
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
import transformers
import sentence_transformers
import numpy as np
import textract
import torch

Matplotlib is building the font cache; this may take a moment.


In [10]:
#Convert PDF to text
doc = textract.process("./Estudio21.pdf")

In [11]:
# Step 2: Save to .txt and reopen (helps prevent issues)
with open("./Estudio21.txt", 'w') as f:
    f.write(doc.decode('utf-8'))

In [12]:
with open('./Estudio21.txt', 'r') as f:
    text = f.read()
text_to_encode = text.split("\n")  # Split by newlines for sentence-like units

In [ ]:
# Load the pre-trained model
model = sentence_transformers.SentenceTransformer("all-mpnet-base-v2")

# Assuming text_to_encode contains document content as a list of sentences
user_question = "Que es HTML5?"
question_embedding = model.encode([user_question])
embeddings = model.encode(text_to_encode)

similarities = model.similarity(question_embedding, embeddings)
most_similar_idx = np.argmax(similarities)

# Consider only sentences after the most similar one
window_size_forward = 4  # Adjust this value as needed
start_idx = most_similar_idx
end_idx = min(len(text_to_encode), most_similar_idx + window_size_forward + 1)

answer_context = " ".join(text_to_encode[start_idx:end_idx])
answer_context_truncated=""
for i in range(0, len(answer_context.split("."))-1):
  answer_context_truncated=answer_context_truncated+" "+answer_context.split(".")[i]
print("Answer:", answer_context_truncated)  # Truncates at the last point

In [13]:
from transformers import BertTokenizer, BertLMHeadModel

# Choose your desired pre-trained model
model_name = "dccuchile/bert-base-spanish-wwm-cased"

# Load tokenizer (handles converting text to numerical representations)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Load the pre-trained model (architecture depends on your choice)
model = BertLMHeadModel.from_pretrained(model_name)  # Adjust for T5 or other models

def summarize(text, max_length=512):

  # Tokenize the input text
  input_ids = tokenizer(text, return_tensors="pt")["input_ids"]

  # Generate summary using the model
  output = model.generate(input_ids, max_length=max_length)

  # Decode the generated tokens back to human-readable text
  summary = tokenizer.decode(output[0], skip_special_tokens=True)
  return summary

  summary = summarize(text, max_length=max_length)

def summarize_long_text(text, max_length=512):
    chunks = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    summaries = [summarize(chunk) for chunk in chunks]
    return " ".join(summaries)

print(f"Original Text: {text}")
print(f"Summary: {summarize_long_text(text)}")

tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

/Users/carlosmontiel/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


Original Text: 1. Lenguajes de Programación:

- Java:

- Fundamentos de Java (sintaxis, variables, operadores, estructuras de control)**: Java

es un lenguaje de programación de alto nivel orientado a objetos, originalmente
desarrollado por Sun Microsystems en 1995 y actualmente propiedad de Oracle. Es
utilizado para desarrollar una amplia gama de aplicaciones de software, desde
aplicaciones móviles hasta aplicaciones de escritorio, web y
juegos[[3](https://www.tutorialspoint.com/java/index.htm)]. La sintaxis de Java es similar a
la de C, lo que significa que si uno aprende la sintaxis de la programación en Java,
automáticamente sería capaz de leer y escribir programas en lenguajes similares como C,
C++ y C#[[1](https://en.wikibooks.org/wiki/Java_Programming/Language_Fundamentals)].
Entre los fundamentos de Java se incluyen las variables, que almacenan datos para su
procesamiento; los operadores, que realizan diferentes tipos de operaciones matemáticas
y lógicas; y las estructuras de c